Workflow automation tools
=========================

Exercise 3 - Running several simulations without recompilation
--------------------------------------------------------------

This page shows how to start a series of simulations using the [BuildingsPy](http://simulationresearch.lbl.gov/modelica/buildingspy/) Python package.

In addition to this package, you need the [Buildings Modelica library](http://simulationresearch.lbl.gov/modelica/) to run it. You can simply copy and paste the code below and run it in your Python environment, or open this notebook in the Jupyter viewer.

Before running the code, the `ppath` variable should point to the location of the Buildings library on your drive.

In [ ]:

#==============================================================================
# Loading the model file
#==============================================================================

# Set the path to the Buildings library on your disk
ppath = 'D:\\path_to_the_buildings_library_on_your_drive'
# Set the path to the model inside the Buildings library
model = 'Buildings.Controls.Continuous.Examples.PIDHysteresis'

#==============================================================================
# Translate the first model
#==============================================================================

import buildingspy.simulate.Simulator as si
s = si.Simulator(model, 'dymola', packagePath = ppath)

s.setSolver('dassl')
s.showGUI(False)
s.setStopTime(86400)
s.setTimeOut(60)
s.translate()

#==============================================================================
# Run a series of simulations with different integrator times
#==============================================================================

from copy import deepcopy
Ti_list = [150, 600, 1200, 2400]

for Ti, i in zip(Ti_list, range(len(Ti_list))):
    
    s_new = deepcopy(s)
    
    outputFileName = 'case'+str(i)
    s_new.setOutputDirectory(outputFileName)
    
    s_new.addParameters({'con.Ti': Ti})
    
    s_new.printModelAndTime()
    s_new.simulate_translated()
    
#==============================================================================
# Read and plot results
#==============================================================================

from buildingspy.io.outputfile import Reader
import os
import shutil
import matplotlib.pyplot as plt

from matplotlib import rc
rc('text', usetex=True)
rc('font', family='serif', size = 14)

plt.figure()
for Ti, i in zip(Ti_list, range(len(Ti_list))):
    
    outputFileName = 'case'+str(i)
    r = Reader(os.path.join(outputFileName,'PIDHysteresis.mat'), 'dymola')
    
    (t, T) = r.values('temSen.T')
    (t, y) = r.values('con.y')
    
    label_Ti = '$T_i = %0.1f $' % Ti
    plt.plot(t/3600, T-273.15, linewidth = 1.5, label = label_Ti)
    
    shutil.rmtree(outputFileName)

plt.xlabel('Time (h)')
plt.ylabel('T (C)')
plt.legend(loc = 'upper left', fontsize = 12)

plt.savefig('ex3_plot.png')

#==============================================================================
# Delete translated directory (optional)
#==============================================================================

s.deleteTranslateDirectory()
